In [11]:
df = spark.read.csv('./data/sales.csv', header=True, inferSchema=True)

In [12]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("sales")

# Q1 Find all distinct countries.

Hint: use select(), distinct()

In [13]:
df.select("Country").distinct().show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



In [14]:
# should be same to this one
distinct_countries = spark.sql("SELECT DISTINCT Country from sales").show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



# Q2 Find the Name and Price of sales records in Brazil.

Hint: use filter().

In [17]:
df.filter(df['Country']=='Brazil').select(df['Name'], df['Price']).show()

+-------+-----+
|   Name|Price|
+-------+-----+
|Joachim| 1200|
|  Diana| 7500|
+-------+-----+



# Q3 For each country, find the total Price.

Hint: Use groupBy()

In [21]:
df.groupBy("Country").sum("Price").show()

+-------------+----------+
|      Country|sum(Price)|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



# Q4 List countries by their total Price in descending order.

Hint: Use orderBy()

In [26]:
df.groupBy("Country").sum("Price").orderBy("sum(Price)", ascending=False).show()

+--------------------+----------+
|             Country|sum(Price)|
+--------------------+----------+
|       United States|    350350|
|      United Kingdom|     63600|
|              Canada|     42000|
|              France|     30300|
|             Ireland|     29100|
|           Australia|     22800|
|             Germany|     22800|
|         Switzerland|     19200|
|         Netherlands|     14400|
|              Norway|     12000|
|              Brazil|      8700|
|             Denmark|      8400|
|              Sweden|      8400|
|        South Africa|      3600|
|             Belgium|      3600|
|             Austria|      3600|
|United Arab Emirates|      3600|
|               Malta|      3600|
|               India|      2400|
|               Italy|      2400|
+--------------------+----------+
only showing top 20 rows



# Q5 Now load a second table 'countries'
http://www.cse.ust.hk/msbd5003/data/countries.csv 

df2 = spark.read.csv('countries.csv', header=True, inferSchema=True)
Redo Question 3, but replace the country names by their IDs.

Hint: Use join()

In [31]:
df2 = spark.read.csv('./data/countries.csv', header=True, inferSchema=True)
df2.show()

+--------------+---+
|       Country| ID|
+--------------+---+
|United Kingdom|  1|
| United States|  2|
|     Australia|  3|
|        Israel|  4|
|        France|  5|
|   Netherlands|  6|
|       Ireland|  7|
|        Canada|  8|
|         India|  9|
|  South Africa| 10|
|       Finland| 11|
|   Switzerland| 12|
|       Denmark| 13|
|       Belgium| 14|
|        Sweden| 15|
|        Norway| 16|
|    Luxembourg| 17|
|         Italy| 18|
|       Germany| 19|
|       Moldova| 20|
+--------------+---+
only showing top 20 rows



In [33]:
df.groupBy("Country").sum("Price").show()

+-------------+----------+
|      Country|sum(Price)|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



In [38]:
joined = df2.join(df, df2.Country == df.Country)
joined.groupBy("ID").sum("Price").show()

+---+----------+
| ID|sum(Price)|
+---+----------+
| 31|      1200|
| 34|      2400|
| 28|      3600|
| 26|      3600|
| 27|      1200|
| 12|     19200|
| 22|      3600|
|  1|     63600|
| 13|      8400|
|  6|     14400|
| 16|     12000|
|  3|     22800|
| 20|      1200|
|  5|     30300|
| 19|     22800|
| 15|      8400|
|  9|      2400|
| 17|      1200|
|  4|      1200|
|  8|     42000|
+---+----------+
only showing top 20 rows



# Q6 Rewrite the PageRank example using DataFrame API.  
Here is a skeleton of the code.  Your job is to fill in the missing part.  The data files can be downloaded at:

https://www.cse.ust.hk/msbd5003/data/pagerank_data.txt

https://www.cse.ust.hk/msbd5003/data/dblp.in

In [128]:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("./data/pagerank_data.txt")
# You can also test your program on the follow larger data set:
#lines = spark.read.text("./data/dblp.in")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))

def test(u, r):
    return u/r

for iteration in range(numOfIterations):    
    t = links.join(outdegrees, 'src')\
    .select('dst', 'src', 'count').join(ranks, 'src')
    t = t.select('*', test(t['rank'], t['count']).alias('slen'))
    
    t = t.withColumnRenamed('dst', 'dst1').groupBy('dst1').sum('slen')
    t = t.withColumnRenamed('dst1', 'src')
    t = t.withColumnRenamed('sum(slen)', 'rank')
    ranks = t

ranks.orderBy(desc('rank')).show()


+---+------------+
|src|        rank|
+---+------------+
|  1|1.3330078125|
|  4|         1.0|
|  3|         1.0|
|  2|0.6669921875|
+---+------------+



In [23]:
links.show()
outdegrees.show()
ranks.show()

+---+---+
|src|dst|
+---+---+
|  1|  2|
|  1|  3|
|  2|  3|
|  3|  4|
|  4|  1|
|  2|  1|
+---+---+

+---+-----+
|src|count|
+---+-----+
|  3|    1|
|  1|    2|
|  4|    1|
|  2|    2|
+---+-----+

+---+----+
|src|rank|
+---+----+
|  3|   1|
|  1|   1|
|  4|   1|
|  2|   1|
+---+----+



Note: There is a bug in the current SparkSQL implementation: The groupBy (followed by some aggregation) method sometimes fails to group all rows with the same key.  A temporary workaround is the following:

Suppose you want to do

In [ ]:
df.groupBy('A').sum('B')

If it fails to produce the desired result, try

In [ ]:
df.withColumnRenamed('A', 'A1').groupBy('A1').sum('B')

We have reported this bug to the Spark developers and the issue is currently under investigation:

https://issues.apache.org/jira/browse/SPARK-20169 (Links to an external site.)